## Multigrid methods

**One of the most powerful acceleration schemes for the convergence of iterative methods in solving elliptic problems is the multigrid algorithm**

**Based on the idea:**
* Different components of the solution converge to the exact solution at different rates --> should be treated differently
* Residual: linear combination of a set of basis vectors --> ranging from smooth to rapidly varying (just like low- and high-frequency sines and cosines):
   * Smooth component of the residual converges very slowly to zero
   * Rough part converges quickly
* The multigrid algorithm takes advantage of this to substantially reduce the overall effort required to obtain a converged solution

**Objective**: 
* Solve 
$$
A\phi = b,
$$
where, $A$ is a matrix obtained from a finite difference approximation to a differential equation.

* Let $\psi = \phi^{(n)}$ be an approximation to the solution $\phi$, which is obtained from an iterative scheme after $n$ iterations
 
* $\mathbf{r}$ is the residual defined as
$$
\mathbf{r} = A\psi - \mathbf{b}
$$

* The residual approaches zero if the approximate solution $\psi$ approaches the exact solution $\phi$
* Subtracting these two equations leads to an equation for the error $\epsilon = \phi − \psi$ in terms of the residual $\mathbf{r}$
$$
A\mathbf{\epsilon} = \mathbf{r},
$$
which is called the residual equation.

* We often talk about driving the residual to zero in our iterative solution process, and we measure the performance of a given solution procedure in terms of the number of iterations required to drive the residual to zero.


### Illustration

Consider the one-dimensional boundary value problem:
$$
\dfrac{d^2 u}{dx^2} = \textrm{sin}(k\pi x) \;\; 0 \le x \le 1 
$$
with BC's: $u(0) = u(1) = 0$.

* Integer k is called the wavenumber and is an indicator of how many oscillations the sine wave would go through in the domain:
   * Higher values of k --> oscillations or “rougher” behavior
* **Exact solution**: $-1/k^2\pi^2 \textrm{sin}(k\pi x)$.
* We pretend we don’t know the exact solution and embark on solving the problem using a finite difference approximation on $N + 1$ uniformly spaced grid points of size $h = 1/N$.
$$
\dfrac{u_{j+1} -2u_j + u_{j-1}}{h^2} = \textrm{sin}(k\pi x_j)
$$
with, $u_0 = u_N = 0$

* Start the itteration with an initial guess: $\mathbf{u}^{(0)} = 0$.
* The first residual is: $r = \textrm{sin}(k\pi_jh)$.
* Use Gauss-Seidel as the basic iteration scheme:
$$
u_j^{(n+1)} = \left[u_{j+1}^{(n)} + u_{j-1}^{(n+1)} - h^2\textrm{sin}(k\pi x_j) \right]
$$
* $n$ is the iteration index

**Evolution of the maximum residual, $\mathbf{r} = \mathbf{b} − Au^{(n)}$, with the number of iterations for different values of wavenumber $\mathbf{k}$** (for N=64):
![Convergence_k](./pic1.png)

* It is clear that the convergence is faster for higher values of $k$. That is, the residual, and hence the error, goes to zero faster for more rapidly varying right-hand sides.

Consider a slightly more complicated right-hand side:
$$
\dfrac{d^2 u}{dx^2} = \dfrac{1}{2}\left[\textrm{sin}(\pi x) + \textrm{sin}(16 \pi x)\right] \;\; 0 \le x \le 1 
$$
with BC's: $u(0) = u(1) = 0$.

**Evolution of the residual as a function of the number of iterations**(for N=64):
![Convergence](./pic2.png)

* Residual goes down rapidly and then it virtually stalls. 
* **This type of convergence history is observed frequently in practice when standard iterative schemes are used.**
    * Rapidly varying part of the residual goes to zero quickly 
    * Smooth part of it remains and diminishes slowly. 
    
**The initial residual, which is the same as the right-hand side of the differential equation, and its profile after 10 and 100 iterations**
![Residual](./pic3.png)
* Clearly only the smooth part of the residual has remained after 100 iterations.

**Key observation of the multigrid algorithm**: 
A slowly varying function on a fine grid would appear as a more rapidly varying function (or rougher) on a coarse grid.

* Get faster convergence on the smooth part of the solution, if we transfer the problem to a coarse grid.
  * Since the smooth part of the solution does not require many grid points to be represented, such a transfer would not cause a large error.

**Strategy**:
* As soon as the convergence of the residual stalls, the iterative process is transferred to a coarse grid
* On the coarse grid, the smooth part of the residual is annihilated faster and cheaper (because of fewer grid points)
* After this is accomplished, one can interpolate the residual back to the fine grid and work on the high wavenumber parts
* This back and forth process between the fine and coarse grids continues until overall convergence is achieved
* In transferring data from fine grid to coarse grid *(called restriction)* we can simply take every other data point. For transfer between coarse and fine grid *(called prolongation)* we can use a straightforward linear interpolation.

### General steps (V cycle)

1. Perform a few iterations on the original equation, $A\phi=b$, on the fine grid with the mesh spacing $h$. Let the resulting solution be denoted by $\psi$. Calculate the residual $r = b− A\psi$ on the same grid.
2. Transfer the residual to acoarse grid **(restriction)** of mesh spacing $2h$, and on this grid iterate on the error equation $A\epsilon = r$, with the initial guess $\epsilon_0 = 0$.
3. Interpolate **(prolongation)** the resulting $\epsilon$ to the fine grid. Make a correction on the previous $\psi$ by adding it to $\epsilon$, i.e., $\psi_{new} =\psi + \epsilon$. Use $\psi_{new}$ as the initial guess to iterate on the original problem, $A\phi = b$.
4. Repeat the process.

Why stop at two levels? 
  * Multiple levels

Three recursive approaches to multigrid are:
![MGM](./pic4.png)

* (a) the recursive algorithm that we just discussed and is referred to as the V cycle
* (b) W cycle
* (c) Full multigrid cycle (FMC): one starts the problem on the coarsest grid and uses the result as the initial condition for the finer mesh and so on. After reaching the finest grid one usually proceeds with the W cycle.

### More on the V-cycle implemented in TD

![VGM](./pic5.png)

**Algorithm** 
![algorithm](./pic6.png)

For the transfer of a quantity q from a fine mesh to a coarse one **"restriction"**, we use:
\begin{eqnarray}
q_{i,j} & = & \frac{1}{4} q_{2i, 2j} + \\
& & \frac{1}{8} q_{2i+1, 2j}  + \frac{1}{8} q_{2i-1, 2j}  + \frac{1}{8} q_{2i, 2j+1} + \frac{1}{8} q_{2i, 2j-1} + \\
& & \frac{1}{16} q_{2i+1, 2j+1}  + \frac{1}{16} q_{2i+1, 2j-1}  + \frac{1}{16} q_{2i-1, 2j+1} + \frac{1}{16} q_{2i-1, 2j-1}
\end{eqnarray}


![restriction](./pic7.png)

For the associated transfer of a quantity q from a coarse to a fine mesh **"prolongation"**, we employ
\begin{eqnarray}
q_{2i,2j} &=& q_{i,j} \\
q_{2i+1,2j} &=& (q_{i,j} + q_{i+1,j} )/2 \\ 
q_{2i,2j+1} &=& (q_{i,j} + q_{i,j+1} )/2 \\ 
q_{2i+1,2j+1} &=& (q_{i,j} + q_{i+1,j} + q_{i,j+1} + q_{i+1,j+1})/4.
\end{eqnarray}

![prolongation](./pic8.png)